In [ ]:
!pip3 install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html
!pip3 install numpy
!pip3 install torch
!pip3 install networkx
!pip3 install matplotlib
!pip install torch_geometric
!pip install fsspec==2024.3.1

In [ ]:
import dgl
import os
import torch
import scipy
import argparse
import seaborn
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch_geometric.datasets import TUDataset
from sklearn.model_selection import train_test_split
from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split
from dgl.data.utils import download, extract_archive, get_download_dir, _get_dgl_url

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
class GraphSAGE(nn.Module):
    '''
    We consider GraphSAGE as Basic GNN module as proposed by Authors.
    You can use other GNN Models like GCN,GAT,GIN etc as well.
    '''
    def __init__(self, infeat, outfeat, device='cpu', use_bn=True, mean=False, add_self=False):
        super().__init__()
        self.add_self = add_self
        self.use_bn = use_bn
        self.device = device
        self.mean = mean
        self.W = nn.Linear(infeat, outfeat, bias=True)
        nn.init.xavier_uniform_(self.W.weight, gain=nn.init.calculate_gain('relu'))

    def forward(self, x, adj):
        if self.add_self:
            adj = adj + torch.eye(adj.size(0)).to(self.device)

        if self.mean:
            adj = adj / adj.sum(1, keepdim=True)

        h_k_N = torch.matmul(adj, x)
        h_k = self.W(h_k_N)
        h_k = F.normalize(h_k, dim=2, p=2)
        h_k = F.relu(h_k)
        if self.use_bn:
            self.bn = nn.BatchNorm1d(h_k.size(1)).to(self.device)
            h_k = self.bn(h_k)
        return h_k

In [ ]:
class DiffPool(nn.Module):
    '''
    Architecture of DiffPool Module.
    '''
    def __init__(self, nfeat, nnext, nhid, device='cpu'):
        super(DiffPool, self).__init__()
        self.device = device
        # GNN to generate embedding
        self.embed = GraphSAGE(nfeat, nhid, device=self.device, use_bn=True)
        # GNN to generate cluster assignment matrix S
        self.assign_mat = GraphSAGE(nfeat, nnext, device=self.device, use_bn=True)

    def forward(self, x, adj, log=False):
        # Equation - 5
        z_l = self.embed(x, adj)

        # Equation - 6
        s_l = F.softmax(self.assign_mat(x, adj), dim=-1)

        # Equation - 3
        xnext = torch.matmul(s_l.transpose(-1, -2), z_l)

        # Equation - 4
        anext = (s_l.transpose(-1, -2)).matmul(adj).matmul(s_l)

        return xnext, anext

In [ ]:
class Model(nn.Module):
    '''
    Architecture of Overall Graph Classifier. As proposed by authors, we apply a DIFFPOOL layer after every two GRAPHSAGE layers
    '''
    def __init__(self, pool_size, device, input_shape, n_classes):
        super().__init__()
        self.input_shape = input_shape
        self.device = device
        self.layers = nn.ModuleList([
            GraphSAGE(input_shape, 30, device=self.device),
            GraphSAGE(30, 30, device=self.device),
            DiffPool(30, pool_size, 30, device=self.device),
            GraphSAGE(30, 30, device=self.device),
            GraphSAGE(30, 30, device=self.device),
        ])
        self.classifier = Classifier(n_classes)

    def forward(self, x, adj):
        for layer in self.layers:
            if isinstance(layer, GraphSAGE):
                x = layer(x, adj)
            elif isinstance(layer, DiffPool):
                x, adj = layer(x, adj)

        readout_x = x.sum(dim=1)
        return readout_x

    def loss(self, output, labels):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, labels)
        return loss

In [ ]:
class Classifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.classifier = nn.Sequential(nn.Linear(30, 50),nn.ReLU(),nn.Linear(50, n_classes))

    def forward(self, x):
        return self.classifier(x)

In [ ]:
class CollateFn:
    def __init__(self, device='cpu'):
        self.device = device

    def __call__(self, batch):
        adj_tensor_list = []
        features_list = []
        mask_list = []
        # (adj, features), labels = list(zip(*batch))
        max_num_nodes = max([item.x.shape[0] for item in batch])
        labels = []
        # for (edge_index, x, y) in batch:
        for d in batch:
            # A Adj F Feature L Label
            A = to_dense_adj(d.edge_index).squeeze(0)
            F = d.x
            # print(F.size())
            L = d.y
            labels.append(L)

            length = F.shape[0]
            pad_len_F = max_num_nodes - F.shape[0]
            pad_len_A = max_num_nodes - A.shape[0]

            adj_tensor_list.append(np.pad(A, ((0, pad_len_A), (0, pad_len_A)), mode='constant'))
            features_list.append(np.pad(F, ((0, pad_len_F), (0, 0)), mode='constant'))
            # mask = np.zeros(max_num_nodes)
            # mask[:length] = 1
            # mask_list.append(mask)
        stacked_adj  = torch.from_numpy(np.stack(adj_tensor_list, 0)).float().to(self.device)
        stacked_feat = torch.from_numpy(np.stack(features_list, 0)).float().to(self.device)
        stacked_y    = torch.from_numpy(np.stack(labels, 0)).long().to(self.device)

        return stacked_adj, stacked_feat, stacked_y

In [ ]:
def main():
    print("Diff Pool Easy Code")
    train_ratio = 0.8
    batch_size = 20
    link_pred = False
    epochs = 1000

    device = "cuda" if torch.cuda.is_available() else "cpu"

    dataset = TUDataset(root='/content/ENZYMES',name = 'ENZYMES')
    dataset_size = len(dataset)
    train_size = int(dataset_size * train_ratio)
    test_size = dataset_size - train_size

    max_num_nodes = max([item.x.shape[0] for item in dataset])
    n_classes = 6
    train_data, test_data = random_split(dataset, (train_size, test_size))
    input_shape = 3
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True,collate_fn=CollateFn(device))
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True,collate_fn=CollateFn(device))

    model = Model(pool_size=6, device=device, input_shape=input_shape, n_classes=n_classes).to(device)

    model.train()
    optimizer = optim.Adam(model.parameters())

    for e in tqdm(range(epochs)):
        # utils.e = e
        epoch_losses_list = []
        true_sample = 0
        model.train()

        for i, (adj, features, batch_labels) in enumerate(train_loader):
        # for data in train_loader:
        #     adj=data.edge_index.to_dense_adj
        #     features = data.x
        #     batch_labels = data.y

            # utils.train_iter += 1
            graph_feat = model(features, adj)
            output = model.classifier(graph_feat)
            batch_labels = batch_labels.squeeze(1)

            # print(batch_labels.size())
            # print(output.size())

            loss = model.loss(output, batch_labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
            optimizer.step()
            optimizer.zero_grad()

            epoch_losses_list.append(loss.item())
            iter_true_sample = (output.argmax(dim=1).long() == batch_labels.long()). \
                float().sum().item()
            iter_acc = float(iter_true_sample) / output.shape[0]
            # utils.writer.add_scalar("iter train acc", iter_acc, utils.train_iter)
            # print(f"{utils.train_iter} iter train acc: {iter_acc}")
            true_sample += iter_true_sample

        acc = true_sample / train_size
        # print(f"Epoch:{e}  \t train_acc:{acc:.2f}")
        print(f"Epoch:{e}, Loss :{np.mean(epoch_losses_list):.4f}, train_acc:{acc:.2f}")
        # print("Loss : ",np.mean(epoch_losses_list))

        test_loss_list = []
        true_sample = 0
        model.eval()
        with torch.no_grad():
            for i, (adj, features, batch_labels) in enumerate(test_loader):
                # utils.test_iter += 1
                graph_feat = model(features, adj)
                batch_labels = batch_labels.squeeze(1)
                output = model.classifier(graph_feat)
                loss = model.loss(output, batch_labels)
                test_loss_list.append(loss.item())
                iter_true_sample = (output.argmax(dim=1).long() == batch_labels.long()). \
                    float().sum().item()
                iter_acc = float(iter_true_sample) / output.shape[0]
                # utils.writer.add_scalar("iter test acc", iter_acc, utils.test_iter)
                # print(f"{utils.test_iter} iter test acc: {iter_acc}")
                true_sample += iter_true_sample
        acc = true_sample / test_size
        # utils.writer.add_scalar("Epoch Acc", acc, e)
        # print(f"Epoch:{e}  \t val_acc:{acc:.2f}")
        print(f"Epoch:{e}, Test_acc:{acc:.2f}")

In [ ]:
main()

Diff Pool Easy Code


Processing...
Done!
/usr/local/lib/python3.10/dist-packages/torch_geometric/io/fs.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location)
  

Epoch:0, Loss :2.0091, train_acc:0.14
Epoch:0, Test_acc:0.20


  0%|          | 2/1000 [00:03<23:54,  1.44s/it]

Epoch:1, Loss :1.7590, train_acc:0.19
Epoch:1, Test_acc:0.25


...

Epoch:997, Loss :0.0364, train_acc:0.99
Epoch:997, Test_acc:0.47


100%|█████████▉| 999/1000 [08:59<00:00,  1.94it/s]

Epoch:998, Loss :0.0472, train_acc:0.99
Epoch:998, Test_acc:0.48


100%|██████████| 1000/1000 [09:00<00:00,  1.85it/s]

Epoch:999, Loss :0.0191, train_acc:0.99
Epoch:999, Test_acc:0.50
